In [2]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [23]:
from email.policy import default
from typing import List  # isort:skip
import os
import re
from operator import itemgetter
from typing import Dict, List
from urllib.parse import urljoin
from collections import Counter
from typing import Dict, Optional

import numpy as np
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from filter_urls import find_articles, find_urls
# from requesting_urls import get_html
from collections import defaultdict
import requests
from numba import jit, types, optional, deferred_type
from numba.experimental import jitclass

session = requests.Session()
headers = {
    "User-Agent": "august.femtehjell@gmail.com Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"
}

def get_html(url: str, params: Optional[Dict] = None, output: Optional[str] = None):
    """Get an HTML page and return its contents.

    Args:
        url (str):
            The URL to retrieve.
        params (dict, optional):
            URL parameters to add.
        output (str, optional):
            (optional) path where output should be saved.
    Returns:
        html (str):
            The HTML of the page, as text.
    """
    # passing the optional parameters argument to the get function
    response = session.get(url, headers=headers)

    html_str = response.text

    if output:
        # if output is specified, the response txt and url get printed to a
        # txt file with the name in `output`
        with open(output, "w") as outfile:
            print(f"Writing to: {output}")
            outfile.write(response.url + "\n")
            outfile.write(html_str)

    return html_str

def better_articles(html):
  base_url = "https://en.wikipedia.org"
  articles = set()
  #print(html)
  # response = requests.get(url, params=None)
  #html_str = response.text
  long_pat = re.compile(r"(?<=href=\")((http\:\/\/en\.wikipedia[^#\\\"]+)|(\/wiki[^#\"]+))")
  
  for article in long_pat.findall(html):
    # print(article)
    # print(url)
    # quit()
    # article = long_pat.search(long_tag)
    if article is not None:
      if (article[0][0] == "/"):
        article = base_url + article[0]
      else:
        article = article[0]
      if not ":" in article[6:]:
        articles.add(article)
  
  return articles

node_type = deferred_type()
spec = [('data', types.unicode_type),
        ('last', optional(node_type)),
        ('next', optional(node_type))]

class Node:
  def __init__(self, data):
    self.data = data
    self.last = None
    self.next = None
  
  def set_prev(self, last):
    self.last = last

# node_type.define(Node.class_type.instance_type)

class LinkedList:
  def __init__(self):
    self.head = None

class Graph:

  def __init__(self, target):
    self.graph = defaultdict(list)
    self.trail = {}
    self.target = target
    #self.ll = LinkedList()
    self.nodes = {}

  def extendGraph(self, node):
    html: str = get_html(node)
    articles = better_articles(html) # Jeg vil at denne linja her skal gå langt utafor
    for article in articles:
      if article != node:
        self.addEdge(node, article)

  def addEdge(self, u, v):
    self.graph[u].append(v)
    
    # print(v in self.trail)
    if not v in self.trail:
      node = Node(v)
      self.trail[v] = node
      #self.nodes.append(Node(v))
      # Node(v).set_prev(self.trail[u])
      node.last = self.trail[u]
    # if u == v:
      # print(u)
    # print(node.data, node.last.data)
    #print(Node(v).last, self.trail[u])
    #print(self.trail[u])
    #print(node.last)

  def Search(self, start):
    # self.nodes.append(Node(start))
    self.trail[start] = Node(start)
    visited = defaultdict(lambda: False)
    # visited = [False] * (max(self.graph) + 1)
    queue = []
    count = 0

    queue.append(start)
    visited[start] = True

    while queue:
      node = queue.pop(0)
      self.extendGraph(node)
      # print(self.trail[node].data)
      # if node == self.target:
      #   print("Found it!")
      #   path = []
      #   breadcrumbs = self.trail[i]
      #   while breadcrumbs.last is not None:
      #     path.insert(0, breadcrumbs.data)
      #     breadcrumbs = breadcrumbs.last
      #   print(path)
      #   return path
      count += 1
      print(count, node)
      for i in self.graph[node]:
        
        # print(count, node, i, self.trail[node].data, self.trail[i].last.data)#self.trail[node])
        if i == self.target:
          print("Found it here!")
          path = [i]
          breadcrumbs = self.trail[i]
          while breadcrumbs.last != None:
            path.insert(0, breadcrumbs.last.data)
            breadcrumbs = breadcrumbs.last
            #print(breadcrumbs.data)
            #print(breadcrumbs.last.data)
            #print(breadcrumbs.last.last.data)
            # new_node = breadcrumbs.last
            #if breadcrumbs == breadcrumbs.last:
            #  break
            # print(breadcrumbs.last, breadcrumbs, new_node)
            # path.insert(0, new_node.data)
            # breadcrumbs = new_node
            # print(_, breadcrumbs.last, breadcrumbs, new_node)
            


          #path = [start] + self.trail[node] + [self.target]
          print(path)
          return path
        if visited[i] == False:
          queue.append(i)
          #print(i, visited[i], self.target)
          visited[i] = True
          




def find_path(start: str, finish: str) -> List[str]:
    """Find the shortest path from `start` to `finish`

    Arguments:
      start (str): wikipedia article URL to start from
      finish (str): wikipedia article URL to stop at

    Returns:
      urls (list[str]):
        List of URLs representing the path from `start` to `finish`.
        The first item should be `start`.
        The last item should be `finish`.
        All items of the list should be URLs for wikipedia articles.
        Each article should have a direct link to the next article in the list.
    """
    #path = [start]
    """
    start_html = get_html(start)
    found = False

    articles = find_articles(start_html)
    print(articles)
    if finish in articles:
      path.append(finish)

    while not found:
      for article in articles:
        article_html = get_html(article)
        new_articles = find_articles(article_html)
        if finish in articles:
          found = True

    """
    graph = Graph(finish)
    path = graph.Search(start)
    assert path[0] == start
    assert path[-1] == finish
    return path

In [25]:
#start = "https://en.wikipedia.org/wiki/Python_(programming_language)"
start = "https://en.wikipedia.org/wiki/Wikipedia"
#finish = "https://en.wikipedia.org/wiki/Peace"
finish = "https://en.wikipedia.org/wiki/Tsar_Bomba"


In [26]:
%lprun -f Graph.Search -f Graph.extendGraph -f get_html -f better_articles find_path(start, finish)

1 https://en.wikipedia.org/wiki/Wikipedia
2 https://en.wikipedia.org/wiki/Glyph
3 https://en.wikipedia.org/wiki/Outline_of_Wikipedia
4 https://en.wikipedia.org/wiki/VDM_Publishing
5 https://en.wikipedia.org/wiki/Bibliography_of_Wikipedia
6 https://en.wikipedia.org/wiki/Engineering_and_Technology_History_Wiki
7 https://en.wikipedia.org/wiki/John_Stossel
8 https://en.wikipedia.org/wiki/Peter_Gabriel
9 https://en.wikipedia.org/wiki/Slate_(magazine)
10 https://en.wikipedia.org/wiki/The_Journal_of_American_History
11 https://en.wikipedia.org/wiki/Open_collaboration
12 https://en.wikipedia.org/wiki/Boris_Tadi%C4%87
13 https://en.wikipedia.org/wiki/Lolicon
14 https://en.wikipedia.org/wiki/CBC_Radio_One
15 https://en.wikipedia.org/wiki/Rigoberta_Mench%C3%BA
16 https://en.wikipedia.org/wiki/Asturian_Wikipedia
17 https://en.wikipedia.org/wiki/PROTECT_IP_Act
18 https://en.wikipedia.org/wiki/ArXiv_(identifier)
19 https://en.wikipedia.org/wiki/Natural_language_processing
20 https://en.wikipedia.org

Timer unit: 1e-06 s

Total time: 557.894 s
File: /var/folders/gk/nd3wqtqx291b3hj52v6ll0qw0000gn/T/ipykernel_36347/2503119746.py
Function: get_html at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                           def get_html(url: str, params: Optional[Dict] = None, output: Optional[str] = None):
    27                                               """Get an HTML page and return its contents.
    28                                           
    29                                               Args:
    30                                                   url (str):
    31                                                       The URL to retrieve.
    32                                                   params (dict, optional):
    33                                                       URL parameters to add.
    34                                                   output (str, optional):
    35                                  

In [13]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
%lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These statis